# Продукты питания в мобильном приложении

## Описание проекта

Существует стартап, который продаёт продукты питания. Нужно разобраться, как ведут себя пользователи мобильного приложения. 

Изучим воронку продаж. Узнаем, как пользователи доходят до покупки. Сколько пользователей доходит до покупки, а сколько — «застревает» на предыдущих шагах? На каких именно?

После этого исследуем результаты A/A/B-эксперимента. Дизайнеры захотели поменять шрифты во всём приложении, а менеджеры испугались, что пользователям будет непривычно. Договорились принять решение по результатам A/A/B-теста. Пользователей разбили на 3 группы: 2 контрольные со старыми шрифтами и одну экспериментальную — с новыми. Выясним, какой шрифт лучше.

Создание двух групп A вместо одной имеет определённые преимущества. Если две контрольные группы окажутся равны, мы можем быть уверены в точности проведенного тестирования. Если же между значениями A и A будут существенные различия, это поможет обнаружить факторы, которые привели к искажению результатов. Сравнение контрольных групп также помогает понять, сколько времени и данных потребуется для дальнейших тестов.


## Описание данных

Каждая запись в логе — это действие пользователя, или событие.

* EventName — название события;
* DeviceIDHash — уникальный идентификатор пользователя;
* EventTimestamp — время события;
* ExpId — номер эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная

## Откроем файл с данными и изучим общую информацию

In [54]:
import pandas as pd
from datetime import datetime, timedelta
import datetime as dt
from matplotlib import pyplot as plt
import numpy as np
import scipy.stats as stats
import math as mth
from plotly import graph_objects as go

In [ ]:
data = pd.read_csv('logs_exp.csv')

data

Вот и первая проблема. Данные склеились в одну строку вместо того, чтобы разбиться по колонкам. Это произошло из-за разделителей в формате csv. csv — это Comma-Separated Values, или значения, разделённые запятыми.

Используем аргумент sep в функции read_csv(), чтобы разделить данные по столбцам,где разделитель будет "\t".

In [ ]:
data = pd.read_csv('logs_exp.csv', sep='\t')
data.head()

## Подготовим данные

### Приведем названия колонок к нижнему регистру.

In [ ]:
data.columns = data.columns.str.lower()

print(data.columns)

### Проверим пропуски и типы данных. Откорректируем, если нужно.

In [ ]:
data.info()

In [ ]:
data.isna().sum()

Пропусков нет.

### Добавим столбец даты и времени, а также отдельный столбец дат.

In [ ]:
data["dateandtime"] = pd.to_datetime(data["eventtimestamp"], unit="s")
data.info()

In [ ]:
data.head()

In [ ]:
data['date'] = pd.to_datetime(data['dateandtime'])
data['date'] = data['date'].dt.floor("D")

In [ ]:
data.head()

Проверим на дубликаты.

In [ ]:
data.duplicated().sum()

Явных дубликатов в таблице datausers по пользователям 413. Удалим их с помощью метода drop_duplicates. И проверим, получилось ли их нам удалить.

In [ ]:
data = data.drop_duplicates()
data.duplicated().sum()

## Изучим и проверим данные.

### Проверим cколько всего событий в логе.

In [ ]:
print(f'Всего событий в логе {len(data)}')
      

### Проверим сколько всего пользователей в логе.

In [ ]:
b=data['deviceidhash'].nunique()
print('Всего пользователей в логе', b)

### Проверим cколько в среднем событий приходится на пользователя.

In [ ]:
print( round(len(data)/data['deviceidhash'].nunique(), 2), 'в среднем событий приходится на пользователя')

### Найдем максимальную и минимальную дату в данных. 

In [ ]:
maxdate=max(data['date'])
mindate=min(data['date'])
print('Максимальная дата:', maxdate)
print('Минимальная дата:', mindate)

### Построем столбчатую диаграмму, которая отобразит количество событий в зависимости от времени в разрезе групп. 

In [ ]:
data.groupby(['expid', 'date'])['date'].count().unstack(0).plot(figsize=(15,10),kind='bar', ylabel='Количество событий', xlabel='Дата',title='Количество событий в зависимости от времени в разрезе групп.')
plt.show()

Технически в логи новых дней по некоторым пользователям могут «доезжать» события из прошлого — это может «перекашивать данные», поэтому удалим данные до 2019-07-31.

In [ ]:
data_old = data.copy()

In [ ]:
data=data[data['date']>'2019-07-31']
data

Проверим, удалились ли данные, построив этот же график заново. Заодно проверим, что у нас есть пользователи из всех трёх экспериментальных групп.

In [ ]:
data.groupby(['expid', 'date'])['date'].count().unstack(0).plot(figsize=(15,10),kind='bar', ylabel='Количество событий', xlabel='Дата',title='Количество событий в зависимости от времени в разрезе групп.')
plt.show()

Пользователи есть во всех трёх экспериментальных группах.Лишние данные удалились.

### Проверим много ли событий и пользователей мы потеряли, отбросив старые данные.

In [ ]:
a=data['deviceidhash'].nunique()
print('Всего пользователей в логе', data['deviceidhash'].nunique())

In [ ]:
print('Потеряли',round((1-a/b)*100, 2), '% пользователей.')

Проверим сколько записей в логе мы потеряли по событиям, в абсолютных и относительных показателях.

In [ ]:
compare=pd.concat([data_old.groupby(['eventname'])['deviceidhash'].count(), data.groupby(['eventname'])['deviceidhash'].count()], axis=1).reset_index()
compare.columns=['eventname', 'old_notes', 'new_notes']
compare['diferent']=compare['old_notes']-compare['new_notes']
compare['share_left']=round(compare['new_notes']/compare['old_notes'], 2)
compare

Записей потеряли не существенно по всем событиям , меньше 1%.


Так же проверим записи по группам.

In [ ]:
compare_expid=pd.concat([data_old.groupby(['expid'])['deviceidhash'].count(), data.groupby(['expid'])['deviceidhash'].count()], axis=1).reset_index()
compare_expid.columns=['eventname', 'old_notes', 'new_notes']
compare_expid['diferent']=compare_expid['old_notes']-compare_expid['new_notes']
compare_expid['share_left']=round(compare_expid['new_notes']/compare_expid['old_notes'], 2)
compare_expid

Записи не существенно потеряли по всем группам , меньше 1%.

## Изучим воронку событий.

### Посмотрим, какие события есть в логах, как часто они встречаются. Отсортируем события по частоте.

In [ ]:
data1=data.groupby('eventname')['date'].count().sort_values(ascending=False)
data1

Чаще всего встречается событие MainScreenAppear.

### Посчитаем, сколько пользователей совершали каждое из этих событий. Отсортируйте события по числу пользователей. Посчитайте долю пользователей, которые хоть раз совершали событие.

In [ ]:
data2=data.groupby('eventname' )['deviceidhash'].nunique().sort_values(ascending=False).to_frame()
data2['share']=round(data2['deviceidhash']/data['deviceidhash'].nunique(),2)
data2


Самая большая доля пользователей у события MainScreenAppear-0.98, а меньше всего у Tutorial-0.11

### Предположим, в каком порядке происходят события. Все ли они выстраиваются в последовательную цепочку? Их не нужно будет учитывать при расчёте воронки.

Судя по таблице data2 можно сделать вывод, что воронка идет, в следующем порядке:

    * MainScreenAppear
    * OffersScreenAppear
    * CartScreenAppear
    * PaymentScreenSuccessful

Так как видно, что на первом этапе больше всего людей,дальше их становится все меньше и меньше.
    
Tutorial не входит в воронуку.

### По воронке событий посчитаем, какая доля пользователей проходит на следующий шаг воронки (от числа пользователей на предыдущем). 

Создадим столбец с предыдущими значениями deviceidhash.

In [ ]:
data3=data2.reset_index().loc[0:3]
data3['deviceidhash2']=data3['deviceidhash'].shift(periods=1)
data3

Заменим первую строку столбца deviceidhash2 на значение deviceidhash, так как это была первая страница, и у нее не было предыдущих значений.

In [ ]:
data3.loc[0, 'deviceidhash2']=data3.loc[0, 'deviceidhash']
data3

Узнаем какая доля пользователей проходит на следующий шаг.

In [ ]:
data3['rate']=round(data3['deviceidhash']/data3['deviceidhash2'], 2)
data3

Самая большая доля пользователей остается на странице PaymentScreenSuccessful -0.95, а меньше всего на странице OffersScreenAppear-0.62.

### Посчитаем сколько людей доходит до последнего шага.

In [ ]:
data3['ratepeople']=data3['deviceidhash2']-data3['deviceidhash']
data3

До оплаты доходит 195 человек из 7419, больше всего людей теряется на странице OffersScreenAppear.

Посчитаем в процентах сколько доходят людей до страницы оплаты.

In [ ]:
print(round(data3.loc[3,'deviceidhash']/data3.loc[0,'deviceidhash']*100, 2), '% доходят людей до страницы оплаты.')

Построим диаграмму воронки.

In [ ]:
fig = go.Figure(go.Funnel(
    y = data3['eventname'],
    x = data3['deviceidhash']))

fig.update_layout(title='Диаграмма воронки')

fig.show()

По воронуе видно, что больше всего людей уходит после перой страницы.

## Изучим результаты эксперимента.

### Посчитаем сколько пользователей в каждой экспериментальной группе.

In [ ]:
data4=data.groupby('expid' )['deviceidhash'].nunique().sort_values(ascending=False).to_frame()
data4

Больше всего пользователей в группе 248, меньше всего в 246.

Проверим, чтобы каждый пользователь принадлежал только одной группе.

In [ ]:
usersdata = data.groupby('deviceidhash')['expid'].nunique().to_frame()
usersdata
us=usersdata.where(usersdata['expid']!=1).count()
us


Каждый пользователь принадлежит только одной группе.

### Есть 2 контрольные группы для А/А-эксперимента, чтобы проверить корректность всех механизмов и расчётов. Проверим, находятся ли статистические критерии разницу между выборками 246 и 247.

In [ ]:
data_expid_eventname = data[data['eventname']!='Tutorial'].groupby(['expid','eventname'])['deviceidhash'].nunique().unstack(0).reset_index().sort_values(by = 246, ascending=False)
data_expid_eventname

Зададим функцию z-теста. Чтобы исключит ошибку первого рода, сделаем поправку Бонферрони и разделим alpha на 16, так как мы проведем 16 экспериментов.

In [ ]:

def ztest(data_expid_eventname, eventname1, expid1, expid2):
    alpha = 0.05/16 # критический уровень статистической значимости


    successes = np.array([data_expid_eventname.loc[eventname1, expid1], data_expid_eventname.loc[eventname1, expid2]])
    trials = np.array([data_expid_eventname[expid1].sum(), data_expid_eventname[expid2].sum()])

    # пропорция успехов в первой группе:
    p1 = successes[0]/trials[0]

    # пропорция успехов во второй группе:
    p2 = successes[1]/trials[1]

    # пропорция успехов в комбинированном датасете:
    p_combined = (successes[0] + successes[1]) / (trials[0] + trials[1])

    # разница пропорций в датасетах
    difference = p1 - p2
    
    # считаем статистику в ст.отклонениях стандартного нормального распределения
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/trials[0] + 1/trials[1]))

    # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = stats.norm(0, 1) 
    
    p_value = round((1 - distr.cdf(abs(z_value))) * 2 ,4)

    print('p-значение: ', p_value)

    if p_value < alpha:
        print('Отвергаем нулевую гипотезу: между группами есть значимая разница')
    else:
        print(
            'Не получилось отвергнуть нулевую гипотезу, нет оснований считать группы разными'
        )

Сранивним группы 246 и 247 всех переходах со страницы на страницу. Сформулируем гипотезы.

Нулевая гипотеза: между группами нет значимой разницы.
Альтернативная гипотеза: между группами есть значимая разница.

Сранивним группы 246 и 247 на странице MainScreenAppear.

In [ ]:
ztest(data_expid_eventname, 1, 246, 247)

Группы 246 и 247 на странице MainScreenAppear одинаковы.

Сранивним группы 246 и 247 на со странице OffersScreenAppear.

In [ ]:
ztest(data_expid_eventname, 2, 246, 247)

Группы 246 и 247 на странице OffersScreenAppear одинаковы.

Сранивним группы 246 и 247 на странице CartScreenAppear.

In [ ]:
ztest(data_expid_eventname, 0, 246, 247)

Группы 246 и 247 на странице CartScreenAppear одинаковы.

Сранивним группы 246 и 247 на странице PaymentScreenSuccessful.

In [ ]:
ztest(data_expid_eventname, 3, 246, 247)

Группы 246 и 247 на странице PaymentScreenSuccessful одинаковы.

Общий вывод: нет оснований группы 246 и 247 считать разными.

### Проверим, находятся ли статистические критерии разницу между выборками 246 и 248.

Сранивним группы 246 и 248 всех переходах со страницы на страницу. Сформулируем гипотезы.

Нулевая гипотеза: между группами нет значимой разницы.
Альтернативная гипотеза: между группами есть значимая разница.

Сранивним группы 246 и 248 на странице MainScreenAppear.

In [ ]:
ztest(data_expid_eventname, 1, 246, 248)

Группы 246 и 248 на странице MainScreenAppear одинаковы.

Сранивним группы 246 и 248 на странице OffersScreenAppear.

In [ ]:
ztest(data_expid_eventname, 2, 246, 248)

Группы 246 и 248 на странице OffersScreenAppear одинаковы.

Сранивним группы 246 и 248 на странице CartScreenAppear.

In [ ]:
ztest(data_expid_eventname, 0,  246, 248)

Группы 246 и 248 на странице CartScreenAppear одинаковы.

Сранивним группы 246 и 248 на странице PaymentScreenSuccessful.

In [ ]:
ztest(data_expid_eventname, 3, 246, 248)

Группы 246 и 248 на странице PaymentScreenSuccessful одинаковы.

Общий вывод: нет оснований группы 246 и 248 считать разными.

### Проверим, находятся ли статистические критерии разницу между выборками 247 и 248.

Сранивним группы 247 и 248 всех переходах со страницы на страницу. Сформулируем гипотезы.

Нулевая гипотеза: между группами нет значимой разницы.
Альтернативная гипотеза: между группами есть значимая разница.

Сранивним группы 247 и 248 на странице MainScreenAppear.

In [ ]:
ztest(data_expid_eventname, 1, 247, 248)

Группы 247 и 248 на странице MainScreenAppear одинаковы.

Сранивним группы 247 и 248 на странице OffersScreenAppear.

In [ ]:
ztest(data_expid_eventname, 2, 247, 248)

Группы 247 и 248 на OffersScreenAppear одинаковы.

Сранивним группы 247 и 248 на CartScreenAppear.

In [ ]:
ztest(data_expid_eventname, 0, 247, 248)

Группы 247 и 248 на CartScreenAppear одинаковы.

Сранивним группы 247 и 248 на странице PaymentScreenSuccessful.

In [ ]:
ztest(data_expid_eventname, 3, 247, 248)

Группы 247 и 248 на странице PaymentScreenSuccessful одинаковы.

Общий вывод: нет оснований группы 247 и 248 считать разными.

### Проверим, находятся ли статистические критерии разницу между выборками (246 + 247) и 248.

In [ ]:
# Создадим новый столбец объединяющий выборки 246 и 247. Назовем столбец 249.
data_expid_eventname[249]=data_expid_eventname[246]+data_expid_eventname[247]
data_expid_eventname

Сранивним группы (246+247) и 248 всех переходах со страницы на страницу. Сформулируем гипотезы.

Нулевая гипотеза: между группами нет значимой разницы. Альтернативная гипотеза: между группами есть значимая разница.

Сранивним группы (246+247) и 248 на странице MainScreenAppear.

In [ ]:
ztest(data_expid_eventname, 1, 249, 248)

Группы (246+247) и 248 на странице MainScreenAppear  одинаковы.

Сранивним группы (246+247) и 248 на странице OffersScreenAppear.

In [ ]:
ztest(data_expid_eventname, 2, 249, 248)

Группы (246+247) и 248 на странице OffersScreenAppear одинаковы.

Сранивним группы (246+247) и 248 на со странице CartScreenAppear.

In [ ]:
ztest(data_expid_eventname, 0, 249, 248)

Группы (246+247) и 248 на странице CartScreenAppear одинаковы.

Сранивним группы (246+247) и 248 на странице PaymentScreenSuccessful.

In [ ]:
ztest(data_expid_eventname, 3, 249, 248)

Группы (246+247) и 248 на странице PaymentScreenSuccessful одинаковы.

Общий вывод: нет оснований группы (246+247) и 248 считать разными.

##  Посчитаем, сколько проверок статистических гипотез вы сделали. Напишем общие выводы.

Мы провели 16 проверок. При проверке групп A1A2, A1B, A2B, (A1+A2) статистически значимых различий между группами не было. Чтобы исключить ошибку первого рода, мы использовали поправку Бонферрони на все тесты. Значит пользователям с новым шрифтом привычно, и его можно оставить.

Группы (246+247) и 248 при переходе со страницы CartScreenAppear на страницу PaymentScreenSuccessful одинаковы.

**Общий вывод: с поправкой Бонферрони, нет статистически значимых различий между граппами с новым шрифтом и старым шрифтом. Значит пользователям с новым шрифтом привычно, и его можно оставить.**